<a href="https://colab.research.google.com/github/dawad/tomato_prediction/blob/master/tomato_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf 
import pathlib 
from tensorflow.keras import datasets, layers, models
import os
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
import csv 
import json
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# def extract images from zip files
import zipfile
def extract_file():
  zip_ref = zipfile.ZipFile("/content/drive/My Drive/assignment_imgs.zip", 'r')
  zip_ref.extractall("/content/drive/My Drive/")
  zip_ref.close()

In [0]:
# read csv file
def read_csv_file(path_file) :
  classes_id =[]
  with open(path_file) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
      for field in row:
        if 'tomat' in field:
          classes_id.append(row[0])
  return classes_id

# read json_file
def read_json_file(path_file, classes_id):
  imges_files =[]
  with open(path_file) as json_file:
    data=json.load(json_file)
    for img_id, value in data.items():
      if value[0]['id'] in classes_id:
        imges_files.append(img_id)
  return imges_files 

In [0]:
# step 3: parse every image in the dataset using `map`
def _parse_function(filename, label):
    image_string = tf.io.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.image.convert_image_dtype(image_decoded, tf.float32)
    image = tf.image.resize(image, [224, 224])
    return image, label

def show(image, label):
  plt.figure()
  plt.imshow(image)
  plt.title(label.numpy().decode('utf-8'))
  plt.axis('off')

In [0]:
# get model of deep learning from tensorflow 
def AlexNet(IMG_HEIGHT, IMG_WIDTH):
  model = models.Sequential()
  # first convolution layer
  model.add(layers.Conv2D(filters=96, kernel_size=(11, 11), strides=(4, 4), padding='valid',
                          input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)))
  model.add(layers.Activation('relu'))
  model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding='valid'))
  model.add(layers.BatchNormalization())
  # second convolution layer
  model.add(layers.Conv2D(filters=256, kernel_size=(11,11), strides=(1,1),
                          padding='valid'))
  model.add(layers.Activation('relu'))
  model.add(layers.MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
  model.add(layers.BatchNormalization())
  # third convolution layer
  model.add(layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
  model.add(layers.Activation('relu'))
  model.add(layers.BatchNormalization())
  # 4th convolution layer
  model.add(layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
  model.add(layers.Activation('relu'))
  model.add(layers.BatchNormalization())
  #5th convolutional layer 
  model.add(layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
  model.add(layers.Activation('relu'))
  model.add(layers.MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
  model.add(layers.Activation('relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.Flatten())
  model.add(layers.Dense(4096, input_shape=(224*224*3,)))
  model.add(layers.Activation('relu'))
  model.add(layers.Dropout(0.4))
  model.add(layers.BatchNormalization())
  model.add(layers.Dense(4096))
  model.add(layers.Activation('relu'))
  model.add(layers.Dropout(0.4))
  model.add(layers.BatchNormalization())
  model.add(layers.Dense(1000))
  model.add(layers.Activation('relu'))
  model.add(layers.Dropout(0.4))
  model.add(layers.BatchNormalization())
  model.add(layers.Dense(2))
  model.add(layers.Activation('softmax'))
  return model

In [0]:
# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [0]:
def train(train_images, labels):
  # step 1: create a dataset returning slices of `filenames`
  dataset = tf.data.Dataset.from_tensor_slices((tf.constant(train_images), tf.constant(labels)))
  dataset = dataset.map(_parse_function)
  dataset = dataset.batch(2)

  # step 2: create iterator and final input tensor
  iterator = dataset.make_one_shot_iterator()
  images_files , labels_id = iterator.get_next()
  epochs = 10
  batch_size = 128
  keep_probability = 0.7
  learning_rate = 0.001
  # step 3: compile AlexNet 
  alexmodel= AlexNet(224,224)
  alexmodel.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  # step 4: train AlexNet 
  history = alexmodel.fit(images_files , labels_id,batch_size=64, epochs=1, verbose=1, 
                          shuffle=True, steps_per_epoch=1000)
  # step 5: save the model
  alexmodel.save('alexmodel.h5')
  return history, alexmodel

In [0]:
# draw accuracy and loss 
def draw_acc_loss(history):
  history_dict = history.history
  acc = history_dict['acc']
  epochs = range(1, len(acc) + 1)
  plt.plot(epochs, acc, 'b', label='accuracy')
  loss = history_dict['loss']
  plt.plot(epochs, loss, 'b', label='loss')
  plt.title('Training and validation loss')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()
  plt.show()

In [0]:
def has_tomato(filename, alexmodel):
  image_string = tf.io.read_file(filename)
  image_decoded = tf.image.decode_jpeg(image_string, channels=3)
  image = tf.image.convert_image_dtype(image_decoded, tf.float32)
  test_image = tf.image.resize(image, [224, 224])
  predictions = alexmodel.predict(tf.reshape(test_image, [1,224,224, 3]), steps=10)
  print(prediction)
  return np.argmax(predictions[0])

In [42]:
# prepare dataset
classes_id = read_csv_file('/content/drive/My Drive/label_mapping.csv')
imges_files = read_json_file('/content/drive/My Drive/img_annotations.json', classes_id)
images = os.listdir('/content/drive/My Drive/assignment_imgs')
train_images =[]
labels =[]
for img in images : 
  if img in imges_files:
    labels.append(1)
  else: 
    labels.append(0)
  train_images.append(os.path.join('/content/drive/My Drive/assignment_imgs', img))
print(len(train_images), len(labels))
# train model 
mod_history, alexmodel = train(train_images, labels)

3000 3000


In [46]:
# test model 
new_model = tf.keras.models.load_model('alexmodel.h5')
path_img = '/content/drive/My Drive/assignment_imgs/ffabf3df9937ea8716c817cacac27346.jpeg'
prediction = has_tomato(path_img, new_model)


0
